# Laminar Bingham Similarity

As usual, we will begin by importing the necessary python packages

In [ ]:
from barennet import SimilarityModel
import tensorflow as tf
import numpy as np

# Uncomment the line below if if you have tensorflow issues regarding your GPU:

# tf.config.set_visible_devices([], 'GPU')

We will find the incomplete similarity exponents for the laminar newtonian 
mean velocity profile in wall coordinates. We know that the dimensionless quantities
involved are $(u^+, \hat{r}, He, Re_\tau)$. They are related through the following equation
outside of the plug region:

$$ u^+ = \frac{Re_\tau}{4} \left( 1 - \hat{r}^2 \right) - \left( 1 - \hat{r} \right) \frac{He}{2 Re_\tau}.$$

We provide a data file generated following the equation above with $10 < Re_\tau < 100$,
 $10 < He < 100$ and $He / Re_\tau^2 < \hat{r} < 1$ (which is exactly the region outside of the plug).
The MDDP construction of the four dimensionless quantities is also provided in the 
code cell below.

In [ ]:
data_path = "../Data/bingham_mvp_data.xlsx"
dimensionally_independent_parameters = ["mu", "rho", "D"]
dimensionally_dependent_parameters = ["r", "tau_y", "- dp/dz"]
dimensional_qoi = "u"
non_dimensional_parameters = ["r^", "He", "Re_tau"]
non_dimensional_qoi = "u+"
non_dimensional_params_construction = {
    "r^": {"r": 1.0, "tau_y": 0, "- dp/dz": 0, "mu": 0, "rho": 0, "D": -1}, 
    "Re_tau": {"r": 0, "- dp/dz": 1/2, "tau_y": 0, "mu": -1, "rho": 0.5, "D": 3/2}, 
    "He": {"r": 0, "tau_y": 1, "- dp/dz": 0, "mu": -2, "rho": 1, "D": 2}
}
non_dimensional_qoi_construction = {
    "u+": {"u": 1, "r": 0, "tau_y": 0, "- dp/dz": -0.5, "mu": 0, "rho": 0.5, "D": -0.5}
}
non_similar_params = ["r^", "He"]
similar_params = ["Re_tau"]

In [ ]:
similarity_model = SimilarityModel(
    data_path=data_path,
    dimensionally_independent_params=dimensionally_independent_parameters,
    dimensionally_dependent_params=dimensionally_dependent_parameters,
    dimensional_qoi=dimensional_qoi,
    non_dimensional_params=non_dimensional_parameters,
    non_dimensional_qoi=non_dimensional_qoi,
    non_dimensional_params_construction=non_dimensional_params_construction,
    non_dimesional_qoi_construction=non_dimensional_qoi_construction,
    non_similar_params=non_similar_params,
    similar_params=similar_params)

### Buckingham's Similarity Group

As soon as the dimensionless construction is provided and we create an instance 
of the SimilarityModel class, the Buckingham's similarity group can be calculated
by solving a linear system (chapter 5 of the thesis).
It shoud read:
\begin{align*}
    &\mu^* = A_1 \mu, \ \ \ \rho^* = A_2 \rho, \ \ \ D^* = A_3 D \\ & \\
    & r^* = A_3 r, \ \ \ \tau_y^* = \frac{A_1^2}{A_2 A_3^2} \tau_y \\ & \\
    & \left( \frac{dp}{dz} \right)^* = \frac{A_1^2}{A_2 A_3^3} \left( \frac{dp}{dz} \right) \\ & \\
    &U^* = \frac{A_1}{A_2 A_3} U
\end{align*}

And, as you can see, this is exactly the output of the cell below.

In [ ]:
similarity_model.print_buckingham_similarity_group()

Remember that we can algebraically manipulate the equation above to see that the laminar MVP fits the definition of incomplete similarity, as a matter of fact:
\begin{equation*}
u^+ = Re_\tau \left( \frac{\left( 1 - \hat{r}^2\right)}{4} - \frac{1 - \hat{r}}{2} \frac{He}{Re_\tau^2} \right) = Re_\tau \Phi^{(1)} \left( \hat{r}, \frac{He}{Re_\tau^2} \right),
\end{equation*}
where $\Phi^{(1)}(z_1, z_2) = (1-z_1) / 4 - (1-z_1)z_2 / 2$. We can now use the find_incomplete_similarity method in order to use the BarenNet to find proper incomplete similarity exponents, as well as estimating the function $\Phi^{(1)}$ with a Deep Neural Network.

In [ ]:
similarity_model.find_incomplete_similarity(n_tries=5, n_epochs=10, tol=5e-3)

After training has been performed, we can check the exponents found by the BarenNet by printing the dictionary used to store them in our class. 

$\cdot$ The entry dict["r^"]["Re_tau"] refers to the exponent of $Re_\tau$ when it is multiplied by $\hat{r}$ i.e. $\xi_3^{(1)}$;

$\cdot$ The entry dict["He"]["Re_tau"] refers to the exponent of $Re_\tau$ when it is multiplied by $He$ i.e. $\xi_3^{(2)}$;

$\cdot$ As one should expect, the entry dict["u+"]["Re_tau"] refers to the exponent of $Re_\tau$ when it is multiplied by $u^+$ i.e. $\xi_1$.

By looking at the incomplete similarity theoretical equation above, we come to the conclusion that $\xi_1 \approx -1,  \ \ \  \xi_3^{(1)} \approx 0$ and $\xi_3^{(2)} \approx -2$. Let's see:

In [ ]:
print(similarity_model.incomplete_similarity_exponents_dict)

Now that incomplete similarity has been found, our package has already automatically calculated the renormalization group from the exponents found. Much like Buckingham's similarity group, we can print it with the following method:

In [ ]:
similarity_model.print_renormalization_group()

Now we will do plot some data points in original coordinates and, afterwards, plot the same points in the renormalized dimensionless coordinates. A data collapse is expected.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
plt.rc('mathtext', fontset="cm")
import matplotlib.ticker as mtick
from matplotlib import cm # Colormaps
import matplotlib.gridspec as gridspec
import seaborn as sns

from tutorial_utils.plotting_methods import (plot_laminar_bingham,
                                             plot_laminar_bingham_renormalized)

In [ ]:
plot_laminar_bingham()

In [ ]:
plot_laminar_bingham_renormalized(similarity_model.incomplete_similarity_exponents_dict)